In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

#magic functions: show plots and change working directory of jupyter notebook
%matplotlib inline
%cd "C:\Users\artemiyburov\prog\tutorials\opencv_tutorial"

c:\Users\artemiyburov\prog\tutorials\opencv_tutorial


In [ ]:
#image read in and grayscaling
img = cv2.imread('lena.jpg',cv2.IMREAD_GRAYSCALE)
cv2.imshow('image',img)
cv2.waitKey(0)
cv2.destroyAllWindows()
plt.imshow(img, cmap = 'gray', interpolation='bicubic')
plt.show()

In [ ]:
#video feed capture and grayscaling
cap = cv2.VideoCapture(0)
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output.avi', fourcc, 20.0, (640,480))
while True:
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    out.write(frame)

    cv2.imshow('frame', frame)
    cv2.imshow('gray', gray)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()

In [ ]:
#thresholding (including gaussian adaptive threshold)
img = cv2.imread('bookpage.jpg')
retval, threshold = cv2.threshold(img, 40, 255, cv2.THRESH_BINARY)

greyscaled = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
retval2, threshold2 = cv2.threshold(greyscaled, 40, 255, cv2.THRESH_BINARY)
gaus = cv2.adaptiveThreshold(greyscaled, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 25, 1)

cv2.imshow('original', img)
cv2.imshow('threshold', threshold)
cv2.imshow('threshold2', threshold2)
cv2.imshow('gaus', gaus)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
#video feed color filtering, smoothing/blurring, morphological transformations
cap = cv2.VideoCapture(0)

while True:
    _, frame = cap.read()
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    lower_red = np.array([0,100,100])
    upper_red = np.array([50,255,255])

    mask = cv2.inRange(hsv, lower_red, upper_red)
    res = cv2.bitwise_and(frame, frame, mask = mask)

    #smoothing
    kernel = np.ones((15,15), np.float32)/225
    smoothed = cv2.filter2D(res, -1, kernel)
    blur = cv2.GaussianBlur(res, (15,15), 0)
    median = cv2.medianBlur(res,15)

    #morphological trafos
    kernel1 = np.ones((5,5), np.uint8)
    erosion = cv2.erode(mask, kernel1, iterations=1)
    dilation = cv2.dilate(mask, kernel1, iterations=1)
    opening = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel1)
    closing = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel1)

    #show results
    cv2.imshow('frame', frame)
    cv2.imshow('mask', mask)
    cv2.imshow('erosion', erosion)
    cv2.imshow('dilation', dilation)
    cv2.imshow('opening', opening)
    cv2.imshow('closing', closing)
    #cv2.imshow('smoothed', smoothed)
    #cv2.imshow('blur', blur)
    #cv2.imshow('median', median)

    #wait for "q" key pressed to exit the loop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

#release the camera and close all windows
cap.release()
cv2.destroyAllWindows()

In [ ]:
#motion-based background extraction (MOG)

cap = cv2.VideoCapture(0)
fgbg = cv2.createBackgroundSubtractorMOG2()

while True:
    _, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    #black = np.zeros(np.shape(frame))
    fgmask = fgbg.apply(gray)
    #mask = np.array([[fgmask], [fgmask], [fgmask]],np.float32)

    filtered = cv2.bitwise_and(frame, frame, mask = fgmask)
    
    cv2.imshow('original', frame)
    cv2.imshow('fgmask', fgmask)
    cv2.imshow('filtered', filtered)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
print(np.shape(frame), np.shape(frame))

cap.release()
cv2.destroyAllWindows()

In [2]:
#haar cascade face and eye detection
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')

cap = cv2.VideoCapture(0)

while True:
    _, img = cap.read()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    for (x,y,w,h) in faces:
        cv2.rectangle(img, (x,y), (x+w, y+h), (255,0,0), 2)
        roi_face_gray = gray[y:y+h, x:x+w]
        roi_face_color = img[y:y+h, x:x+w]
        eyes = eye_cascade.detectMultiScale(roi_face_gray)
        for (ex,ey,ew,eh) in eyes:
            cv2.rectangle(img[y:y+h, x:x+w], (ex,ey), (ex+ew,ey+eh), (0,255,0), 2)

    cv2.imshow('img', img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()